<a href="https://colab.research.google.com/github/revs1/Janatahack---AV-Hackathon-Topic-Modelling/blob/master/Topic_Modelling_Using_LDA_Train_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn import decomposition 
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df = pd.read_csv('/content/sample_data/train.csv')

In [ ]:
df

,ID,TITLE,ABSTRACT,Computer Science,Physics,Mathematics,Statistics,Quantitative Biology,Quantitative Finance
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,1,0,0,0,0,0
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,1,0,0,0,0,0
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,0,0,1,0,0,0
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,0,0,1,0,0,0
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
20967,20968,Contemporary machine learning: a guide for pra...,Machine learning is finding increasingly bro...,1,1,0,0,0,0
20968,20969,Uniform diamond coatings on WC-Co hard alloy c...,Polycrystalline diamond coatings have been g...,0,1,0,0,0,0
20969,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,1,0,0,0,0,0
20970,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,0,0,1,1,0,0


In [ ]:
# Converting binary column to category
to_convert = ['Computer Science', 'Physics', 'Mathematics','Statistics', 'Quantitative Biology', 'Quantitative Finance']

# Make a copy of train data
topic_data = df.copy()

In [ ]:
# Changing the binary fields to categorical fields
topic_data = topic_data[topic_data[to_convert]==1].stack().reset_index().drop(0,1)

topic_data['ID'] = topic_data['level_0'].apply(lambda x: x+1)
topic_data = topic_data.drop('level_0', axis=1)

# Merge the data based on ID
merge_data = df.merge(topic_data, how='left', on='ID' )
# Drop all the binary fields
merge_data = merge_data.drop(to_convert, axis=1)

In [ ]:
# Rename the column to Category
merge_data = merge_data.rename({'level_1':'CATEGORY'}, axis=1)
merge_data

,ID,TITLE,ABSTRACT,CATEGORY
0,1,Reconstructing Subject-Specific Effect Maps,Predictive models allow subject-specific inf...,Computer Science
1,2,Rotation Invariance Neural Network,Rotation invariance and translation invarian...,Computer Science
2,3,Spherical polyharmonics and Poisson kernels fo...,We introduce and develop the notion of spher...,Mathematics
3,4,A finite element approximation for the stochas...,The stochastic Landau--Lifshitz--Gilbert (LL...,Mathematics
4,5,Comparative study of Discrete Wavelet Transfor...,Fourier-transform infra-red (FTIR) spectra o...,Computer Science
...,...,...,...,...
26262,20970,Analysing Soccer Games with Clustering and Con...,We present a new approach for identifying si...,Computer Science
26263,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,Mathematics
26264,20971,On the Efficient Simulation of the Left-Tail o...,The sum of Log-normal variates is encountere...,Statistics
26265,20972,Why optional stopping is a problem for Bayesians,"Recently, optional stopping has been a subje...",Mathematics


In [ ]:
merge_data['CATEGORY'].value_counts()

Computer Science        8594
Physics                 6013
Mathematics             5618
Statistics              5206
Quantitative Biology     587
Quantitative Finance     249
Name: CATEGORY, dtype: int64

In [ ]:
df = merge_data.copy()
df.columns


Index(['ID', 'TITLE', 'ABSTRACT', 'CATEGORY'], dtype='object')

In [ ]:
pd.set_option('display.max_colwidth',-1)
df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,ID,TITLE,ABSTRACT,CATEGORY
0,1,Reconstructing Subject-Specific Effect Maps,"Predictive models allow subject-specific inference when analyzing disease\nrelated alterations in neuroimaging data. Given a subject's data, inference can\nbe made at two levels: global, i.e. identifiying condition presence for the\nsubject, and local, i.e. detecting condition effect on each individual\nmeasurement extracted from the subject's data. While global inference is widely\nused, local inference, which can be used to form subject-specific effect maps,\nis rarely used because existing models often yield noisy detections composed of\ndispersed isolated islands. In this article, we propose a reconstruction\nmethod, named RSM, to improve subject-specific detections of predictive\nmodeling approaches and in particular, binary classifiers. RSM specifically\naims to reduce noise due to sampling error associated with using a finite\nsample of examples to train classifiers. The proposed method is a wrapper-type\nalgorithm that can be used with different binary classifiers in a diagnostic\nmanner, i.e. without information on condition presence. Reconstruction is posed\nas a Maximum-A-Posteriori problem with a prior model whose parameters are\nestimated from training data in a classifier-specific fashion. Experimental\nevaluation is performed on synthetically generated data and data from the\nAlzheimer's Disease Neuroimaging Initiative (ADNI) database. Results on\nsynthetic data demonstrate that using RSM yields higher detection accuracy\ncompared to using models directly or with bootstrap averaging. Analyses on the\nADNI dataset show that RSM can also improve correlation between\nsubject-specific detections in cortical thickness data and non-imaging markers\nof Alzheimer's Disease (AD), such as the Mini Mental State Examination Score\nand Cerebrospinal Fluid amyloid-$\beta$ levels. Further reliability studies on\nthe longitudinal ADNI dataset show improvement on detection reliability when\nRSM is used.\n",Computer Science
1,2,Rotation Invariance Neural Network,"Rotation invariance and translation invariance have great values in image\nrecognition tasks. In this paper, we bring a new architecture in convolutional\nneural network (CNN) named cyclic convolutional layer to achieve rotation\ninvariance in 2-D symbol recognition. We can also get the position and\norientation of the 2-D symbol by the network to achieve detection purpose for\nmultiple non-overlap target. Last but not least, this architecture can achieve\none-shot learning in some cases using those invariance.\n",Computer Science
2,3,Spherical polyharmonics and Poisson kernels for polyharmonic functions,"We introduce and develop the notion of spherical polyharmonics, which are a\nnatural generalisation of spherical harmonics. In particular we study the\ntheory of zonal polyharmonics, which allows us, analogously to zonal harmonics,\nto construct Poisson kernels for polyharmonic functions on the union of rotated\nballs. We find the representation of Poisson kernels and zonal polyharmonics in\nterms of the Gegenbauer polynomials. We show the connection between the\nclassical Poisson kernel for harmonic functions on the ball, Poisson kernels\nfor polyharmonic functions on the union of rotated balls, and the Cauchy-Hua\nkernel for holomorphic functions on the Lie ball.\n",Mathematics
3,4,A finite element approximation for the stochastic Maxwell--Landau--Lifshitz--Gilbert system,"The stochastic Landau--Lifshitz--Gilbert (LLG) equation coupled with the\nMaxwell equations (the so called stochastic MLLG system) describes the creation\nof domain walls and vortices (fundamental objects for the novel nanostructured\nmagnetic memories). We first reformulate the stochastic LLG equation into an\nequation with time-differentiable solutions. We then propose a convergent\n$\theta$-linear scheme to approximate the solutions of the reformulated system.\nAs a consequence, we prove convergence of the approximate so

In [ ]:
X_train,X_hold = train_test_split(df,test_size=0.6,random_state = 111)

In [ ]:
X_train['CATEGORY'].value_counts()

Computer Science        3456
Physics                 2353
Mathematics             2242
Statistics              2139
Quantitative Biology    224 
Quantitative Finance    92  
Name: CATEGORY, dtype: int64

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if (len(word)>3 and len(word.strip('Xx/'))>2)]
  #stems = [stemmer.stem(item) for item in tokens]
  return tokens

In [ ]:
vectorizer_tf = TfidfVectorizer(tokenizer= tokenize,stop_words='english',max_df=0.75,min_df=50,max_features=10000,use_idf=False,norm=None)
tf_vectors = vectorizer_tf.fit_transform(X_train.ABSTRACT)

In [ ]:
tf_vectors.A

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectorizer_tf.get_feature_names()

['-dimensional',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '\\alpha',
 '\\beta',
 '\\cite',
 '\\delta',
 '\\emph',
 '\\epsilon',
 '\\frac',
 '\\gamma',
 '\\geq',
 '\\infty',
 '\\lambda',
 '\\ldots',
 '\\leq',
 '\\log',
 '\\mathbb',
 '\\mathcal',
 '\\mathrm',
 '\\omega',
 '\\right',
 '\\sigma',
 '\\sim',
 '\\sqrt',
 '\\textit',
 '\\theta',
 '\\times',
 'ability',
 'able',
 'absence',
 'absolute',
 'absorption',
 'abstract',
 'accelerated',
 'acceleration',
 'access',
 'according',
 'account',
 'accounts',
 'accuracy',
 'accurate',
 'accurately',
 'achievable',
 'achieve',
 'achieved',
 'achieves',
 'achieving',
 'acoustic',
 'acquired',
 'acquisition',
 'action',
 'actions',
 'activation',
 'active',
 'activities',
 'activity',
 'actual',
 'actually',
 'adapt',
 'adaptation',
 'adapted',
 'adaptive',
 'added',
 'adding',
 'addition',
 'additional',
 'additionally',
 'additive',
 'address',
 'addressed',
 'addresses',
 'addressing',
 'admit',
 'admits',
 'adopt',
 'adopted',
 'advanc

In [ ]:
lda = decomposition.LatentDirichletAllocation(n_components=6,max_iter=3,learning_method='online',learning_offset=50,n_jobs=1,random_state=111)

W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_ 

In [ ]:
W1

array([[0.4828631 , 0.10266479, 0.29767638, 0.113707  , 0.00154643,
        0.0015423 ],
       [0.00506452, 0.9746415 , 0.00506283, 0.00506728, 0.00507799,
        0.00508588],
       [0.94230384, 0.00476887, 0.00479106, 0.00478408, 0.004815  ,
        0.03853714],
       ...,
       [0.00364935, 0.83559348, 0.00365458, 0.00367364, 0.00366272,
        0.14976624],
       [0.00700834, 0.00699229, 0.00700247, 0.00698802, 0.6965687 ,
        0.27544018],
       [0.00429637, 0.87271948, 0.11008643, 0.00429068, 0.00430344,
        0.0043036 ]])

In [ ]:
num_words = 20

vocab = np.array(vectorizer_tf.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [ ]:
topics

['data model paper different analysis information using network results systems time used networks based research social study approach users work',
 'prove problem paper study results graph \\mathbb result space functions \\mathcal function number graphs group given bound case finite theorem',
 'learning network neural networks data model deep training method models methods performance using propose approach machine proposed classification paper based',
 'algorithm data problem model method distribution proposed models based optimal methods paper algorithms approach estimation random time optimization results number',
 'systems quantum energy theory states equations equation numerical model flow method dynamics solutions results nonlinear order using time scheme boundary',
 'phase model field magnetic using temperature density results mass high observed surface energy state properties range different effect study effects']

In [ ]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_train.ABSTRACT))]

df_doc_topic = pd.DataFrame(np.round(W1,2),columns=colnames,index=docnames)
significanttopic = np.argmax(df_doc_topic.values,axis=1)

df_doc_topic['dominant_topic'] = significanttopic

In [ ]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.48,0.10,0.30,0.11,0.00,0.00,0
Doc1,0.01,0.97,0.01,0.01,0.01,0.01,1
Doc2,0.94,0.00,0.00,0.00,0.00,0.04,0
Doc3,0.00,0.00,0.00,0.98,0.00,0.00,3
Doc4,0.00,0.31,0.00,0.55,0.00,0.14,3
...,...,...,...,...,...,...,...
Doc10501,0.50,0.00,0.25,0.11,0.00,0.14,0
Doc10502,0.01,0.89,0.09,0.01,0.01,0.01,1
Doc10503,0.00,0.84,0.00,0.00,0.00,0.15,1
Doc10504,0.01,0.01,0.01,0.01,0.70,0.28,4


In [ ]:
X_train.head()

,ID,TITLE,ABSTRACT,CATEGORY
12536,9998,"Network-based methods for outcome prediction in the ""sample space""","In this thesis we present the novel semi-supervised network-based algorithm\nP-Net, which is able to rank and classify patients with respect to a specific\nphenotype or clinical outcome under study. The peculiar and innovative\ncharacteristic of this method is that it builds a network of samples/patients,\nwhere the nodes represent the samples and the edges are functional or genetic\nrelationships between individuals (e.g. similarity of expression profiles), to\npredict the phenotype under study. In other words, it constructs the network in\nthe ""sample space"" and not in the ""biomarker space"" (where nodes represent\nbiomolecules (e.g. genes, proteins) and edges represent functional or genetic\nrelationships between nodes), as usual in state-of-the-art methods. To assess\nthe performances of P-Net, we apply it on three different publicly available\ndatasets from patients afflicted with a specific type of tumor: pancreatic\ncancer, melanoma and ovarian cancer dataset, by using the data and following\nthe experimental set-up proposed in two recently published papers [Barter et\nal., 2014, Winter et al., 2012]. We show that network-based methods in the\n""sample space"" can achieve results competitive with classical supervised\ninductive systems. Moreover, the graph representation of the samples can be\neasily visualized through networks and can be used to gain visual clues about\nthe relationships between samples, taking into account the phenotype associated\nor predicted for each sample. To our knowledge this is one of the first works\nthat proposes graph-based algorithms working in the ""sample space"" of the\nbiomolecular profiles of the patients to predict their phenotype or outcome,\nthus contributing to a novel research line in the framework of the Network\nMedicine.\n",Statistics
22687,18103,Existence and nonexistence of positive solutions to some fully nonlinear equation in one dimension,"In this paper, we consider the existence (and nonexistence) of solutions to\n\[\n-\mathcal{M}_{\lambda,\Lambda}^\pm (u'') + V(x) u = f(u) \quad {\rm in} \\n\mathbf{R}\n\] where $\mathcal{M}_{\lambda,\Lambda}^+$ and\n$\mathcal{M}_{\lambda,\Lambda}^-$ denote the Pucci operators with $0< \lambda\n\leq \Lambda < \infty$, $V(x)$ is a bounded function, $f(s)$ is a continuous\nfunction and its typical example is a power-type nonlinearity $f(s)\n=|s|^{p-1}s$ $(p>1)$. In particular, we are interested in positive solutions\nwhich decay at infinity, and the existence (and nonexistence) of such solutions\nis proved.\n",Mathematics
24588,19647,Doing Things Twice (Or Differently): Strategies to Identify Studies for Targeted Validation,"The ""reproducibility crisis"" has been a highly visible source of scientific\ncontroversy and dispute. Here, I propose and review several avenues for\nidentifying and prioritizing research studies for the purpose of targeted\nvalidation. Of the various proposals discussed, I identify scientific data\nscience as being a strategy that merits greater attention among those\ninterested in reproducibility. I argue that the tremendous potential of\nscientific data science for uncovering high-value research studies is a\nsignificant and rarely discussed benefit of the transition to a fully\nopen-access publishing model.\n",Computer Science
22215,17727,Shrinkage Estimation Strategies in Generalized Ridge Regression Models Under Low/High-Dimension Regime,"In this study, we propose shrinkage methods based on {\it generalized ridge\nregression} (GRR) estimation which is suitable for both multicollinearity and\nhigh dimensional problems with small number of samples (large $p$, small $n$).\nAlso, it is obtained theoretical properties of the proposed estimators for\nLow/High Dimensional cases. Furthermore, the performance of the listed\nestimators is demonstrated by both simulation studies and real-data analysis,\nand co

In [ ]:
WHold = lda.transform(vectorizer_tf.transform(X_hold.ABSTRACT[:5]))

In [ ]:
colnames = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(X_hold.ABSTRACT[:5]))]

df_doc_topics = pd.DataFrame(np.round(WHold,2),columns=colnames,index=docnames)
significanttopic = np.argmax(df_doc_topics.values,axis=1)

df_doc_topics['dominant_topic'] = significanttopic

In [ ]:
df_doc_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.36,0.00,0.10,0.54,0.0,0.00,3
Doc1,0.00,0.00,0.12,0.87,0.0,0.00,3
Doc2,0.23,0.05,0.37,0.32,0.0,0.03,2
Doc3,0.00,0.00,0.99,0.00,0.0,0.00,2
Doc4,0.75,0.15,0.00,0.10,0.0,0.00,0


In [ ]:
X_hold.head()

,ID,TITLE,ABSTRACT,CATEGORY
9439,7492,In Search of an Entity Resolution OASIS: Optimal Asymptotic Sequential Importance Sampling,"Entity resolution (ER) presents unique challenges for evaluation methodology.\nWhile crowdsourcing platforms acquire ground truth, sound approaches to\nsampling must drive labelling efforts. In ER, extreme class imbalance between\nmatching and non-matching records can lead to enormous labelling requirements\nwhen seeking statistically consistent estimates for rigorous evaluation. This\npaper addresses this important challenge with the OASIS algorithm: a sampler\nand F-measure estimator for ER evaluation. OASIS draws samples from a (biased)\ninstrumental distribution, chosen to ensure estimators with optimal asymptotic\nvariance. As new labels are collected OASIS updates this instrumental\ndistribution via a Bayesian latent variable model of the annotator oracle, to\nquickly focus on unlabelled items providing more information. We prove that\nresulting estimates of F-measure, precision, recall converge to the true\npopulation values. Thorough comparisons of sampling methods on a variety of ER\ndatasets demonstrate significant labelling reductions of up to 83% without loss\nto estimate accuracy.\n",Statistics
10785,8583,Subspace Clustering with Missing and Corrupted Data,"Given full or partial information about a collection of points that lie close\nto a union of several subspaces, subspace clustering refers to the process of\nclustering the points according to their subspace and identifying the\nsubspaces. One popular approach, sparse subspace clustering (SSC), represents\neach sample as a weighted combination of the other samples, with weights of\nminimal $\ell_1$ norm, and then uses those learned weights to cluster the\nsamples. SSC is stable in settings where each sample is contaminated by a\nrelatively small amount of noise. However, when there is a significant amount\nof additive noise, or a considerable number of entries are missing, theoretical\nguarantees are scarce. In this paper, we study a robust variant of SSC and\nestablish clustering guarantees in the presence of corrupted or missing data.\nWe give explicit bounds on amount of noise and missing data that the algorithm\ncan tolerate, both in deterministic settings and in a random generative model.\nNotably, our approach provides guarantees for higher tolerance to noise and\nmissing data than existing analyses for this method. By design, the results\nhold even when we do not know the locations of the missing data; e.g., as in\npresence-only data.\n",Statistics
12291,9804,A Brief Introduction to Machine Learning for Engineers,"This monograph aims at providing an introduction to key concepts, algorithms,\nand theoretical results in machine learning. The treatment concentrates on\nprobabilistic models for supervised and unsupervised learning problems. It\nintroduces fundamental concepts and algorithms by building on first principles,\nwhile also exposing the reader to more advanced topics with extensive pointers\nto the literature, within a unified notation and mathematical framework. The\nmaterial is organized according to clearly defined categories, such as\ndiscriminative and generative models, frequentist and Bayesian approaches,\nexact and approximate inference, as well as directed and undirected models.\nThis monograph is meant as an entry point for researchers with a background in\nprobability and linear algebra.\n",Statistics
13040,10396,Building Robust Deep Neural Networks for Road Sign Detection,"Deep Neural Networks are built to generalize outside of training set in mind\nby using techniques such as regularization, early stopping and dropout. But\nconsiderations to make them more resilient to adversarial examples are rarely\ntaken. As deep neural networks become more prevalent in mission-critical and\nreal-time systems, miscreants start to attack them by intentionally making deep\nneural networks to misclassify an object of one ty